## Final Project: Reddit 
We expect that by the time you complete your check-in, you should have at least selected your dataset and question/problem that you are looking to answer/solve and begun preprocessing and manipulating the data.

Question/problem: Can we predict a user's age based off of characteristics of their submission?

In [13]:
import pandas as pd
import praw

In [16]:
# scrape the teenager subreddit posts

if __name__ == '__main__':

    reddit = praw.Reddit(client_id = '6AM4B2tSyxBJSwue1qcfIg',client_secret = 'eSfVhPV45iW2e2YqzlqSO6EC5ME3GA',user_agent = '1998 ML Project')

    teenager_subreddit = reddit.subreddit('Teenagers')

    id_list = []
    upvotes = []
    authors = []
    post_flairs = []
    user_flairs = []
    titles = []
    num_comments = []
    distinguished = []
    over_18 = []

    for submission in teenager_subreddit.new(limit = 100000):
        upvotes.append(submission.score)
        authors.append(submission.author)
        post_flairs.append(submission.link_flair_text)
        user_flairs.append(submission.author_flair_text)
        titles.append(submission.title)
        num_comments.append(submission.num_comments)
        distinguished.append(submission.distinguished)
        over_18.append(submission.over_18)
        id_list.append(submission.id)




    print("Done")

    df = pd.DataFrame({
        'id':id_list,
        "author":authors,
        'title':titles,
        'num_of_comments':num_comments,
        'distinguished':distinguished,
        'over_18':over_18,
        'post_flairs':post_flairs,
        'author_flairs':user_flairs,
        'upvotes':upvotes
        })

    df.to_csv('reddit_dataset.csv', index=True)

Done


In [19]:
# initial dataset—uncleaned

reddit_df = pd.read_csv('/Users/indiachina/Desktop/INFO 1998/Final Project/reddit_dataset.csv',encoding = 'unicode_escape')

,Unnamed: 0,id,author,title,num_of_comments,distinguished,over_18,post_flairs,author_flairs,upvotes
983,983,qjsdev,broji04,1001 comments and I'll confess to me crush but...,3,NaN,False,Advice,17,2
984,984,qjscn8,bigbootynikee,anyone wanna snap :3,8,NaN,False,Social,NaN,1
985,985,qjscfg,dauntlessdefiance,I want a skinny emo boyfriend,21,NaN,False,Other,16,1
986,986,qjscc6,Nintendocat64,"Swedish fish has given us power, make cursed m...",0,NaN,False,Meme,13,1
987,987,qjs9x0,BottleSome2751,Fucking kill me,4,NaN,False,Meme,16,23


In [20]:
# cleaning dataset: drop NaNs from author_flairs and flairs that say "2 MILLION ATTENDEE"

reddit_df = reddit_df.dropna(subset=["author_flairs"])
reddit_df = reddit_df[reddit_df["author_flairs"] != "2 MILLION ATTENDEE"]
reddit_df.head()
reddit_df.shape[0]

# another column: swear words?

603